# Gender Classification using Facial Images

#### Import Statements

In [1]:
import numpy as np 
import plotly.express as px 
import pandas as pd 
import cv2 
import os 
from glob import glob
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Activation, Dropout, Flatten, Dense, Dropout, LayerNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import matplotlib.pyplot as plt
import pickle 

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

#added import for sgd
from tensorflow.keras.optimizers import SGD

# for Landmark detection
# import dlib
import dlib

import dlib

from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb

import imutils

from tensorflow.keras.metrics import Precision, Recall

e:\virtualenv\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Date Retrival and Cleaning

#### Converting the txt data to a dataframe

In [2]:
df_list = []
for file_name in glob("/age and gender prediction/project/data/AdienceGender/text reference to images/*.txt"):
    df_temp = pd.read_csv(file_name, sep="\t")
    df_list.append(df_temp)
df = pd.concat(df_list, axis=0, ignore_index=True)
del df_list

In [3]:
df.shape

(19370, 12)

#### Dropping null values

In [4]:
df = df.dropna()

In [5]:
df.shape

(18591, 12)

#### Using the details of dataframe to get the image path

In [6]:
df['image_path'] = df[['user_id', 'face_id', 'original_image']].apply(
    lambda x: os.path.join('/age and gender prediction/project/data/AdienceGender/aligned', f"{x[0]}", f"landmark_aligned_face.{x[1]}.{x[2]}"), axis=1)

#### Eliminating images with gender as *unidentified*

In [7]:
new_df = df[df['gender'] != 'u'][['age', 'gender', 'x', 'y', 'dx', 'dy','image_path']]

In [8]:
del df

#### Mapping target to float value

In [9]:
new_df['gender'] = new_df['gender'].apply(lambda x : 1 if x == 'm' else 0).astype(np.float32)

In [10]:
new_df.gender.value_counts()

0.0    9372
1.0    8120
Name: gender, dtype: int64

## Image Preprocessing

#### Preparing to split for train and test set

In [11]:
X = new_df[['image_path']].values 
y = new_df[['gender']].values 

#### Train Test Split

In [12]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
y_train

array([[1.],
       [0.],
       [1.],
       ...,
       [0.],
       [0.],
       [1.]], dtype=float32)

#### Assigning uniform image extensions and resizing 

In [13]:
def preprocess_image(individual_path):
    img = tf.io.read_file(np.array(individual_path).ravel()[0]) 
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [227,227])
    return img 

In [14]:
X_train

array([['/age and gender prediction/project/data/AdienceGender/aligned\\10328235@N07\\landmark_aligned_face.1178.10507385625_bb4f7c957b_o.jpg'],
       ['/age and gender prediction/project/data/AdienceGender/aligned\\10280355@N07\\landmark_aligned_face.1869.9496769730_46a8129a5c_o.jpg'],
       ['/age and gender prediction/project/data/AdienceGender/aligned\\37303189@N08\\landmark_aligned_face.84.10601084834_248e421f0c_o.jpg'],
       ...,
       ['/age and gender prediction/project/data/AdienceGender/aligned\\20632896@N03\\landmark_aligned_face.558.9825344995_b7a2112177_o.jpg'],
       ['/age and gender prediction/project/data/AdienceGender/aligned\\20254529@N04\\landmark_aligned_face.43.9403272379_c104c8cd99_o.jpg'],
       ['/age and gender prediction/project/data/AdienceGender/aligned\\16886060@N03\\landmark_aligned_face.1914.11378844713_467a22d87b_o.jpg']],
      dtype=object)

## Face Detection, Face Alignment, Landmark Detection, and Image Denoising

#### Detecting faces using cv2's face_cascade function, and overwriting with the original image

In [16]:
def yield_training_values(X_train,y_train):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    fa = FaceAligner(predictor, desiredFaceWidth=256)
    
    for image_path, value in zip(X_train, y_train):
        imageP = image_path[0].decode("utf-8")
        img= cv2.imread(imageP, 1)
        denoised_image = cv2.fastNlMeansDenoisingColored(img, None, 5, 6, 7, 21)

        gray = cv2.cvtColor(denoised_image, cv2.COLOR_BGR2GRAY)
        # Detect the face
        rects = detector(gray, 1)
        # Detect landmarks for each face
        try: 
            for rect in rects:
                faceAligned = fa.align(img, gray, rect)

            gray1 = cv2.cvtColor(faceAligned, cv2.COLOR_BGR2GRAY)
            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            faces = face_cascade.detectMultiScale(gray1, 1.3, 5)

            try:
                for (x,y,w,h) in faces:
                    # for putting rectangle on face
                    # cv2.rectangle(faceAligned, (x,y), (x+w, y+h), (0, 255, 0),3)
                    roi_color = faceAligned[y:y+h, x:x+w]
                    cv2.imwrite(imageP , roi_color)
            except:
                continue
        except:
            continue
        
        image = preprocess_image([bytes(imageP, 'utf-8')])
        yield image, value 

#### Using train and test for the model

In [17]:
ds_train = tf.data.Dataset.from_generator(yield_training_values,
                                          args=[X_train, y_train],
                                          output_types=(tf.float32, tf.float32),
                                          output_shapes=([227, 227, 3], [1]))


ds_test = tf.data.Dataset.from_generator(yield_training_values,
                                          args=[X_test, y_test],
                                          output_types=(tf.float32, tf.float32),
                                          output_shapes=([227, 227, 3], [1]))

### Shuffling the data

In [18]:
AUTOTUNE = tf.data.AUTOTUNE
ds_train = ds_train.cache().shuffle(buffer_size=1000).batch(32).prefetch(buffer_size=AUTOTUNE)
ds_test = ds_test.cache().shuffle(buffer_size=1000).batch(32).prefetch(buffer_size=AUTOTUNE)

## Model Implementation

### Data Augmentation

In [19]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(0.2,0.2),
])

### Model Building

In [20]:
model = keras.models.Sequential([
    data_augmentation,
    keras.layers.Conv2D(filters=96, kernel_size=(7,7), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=((2,2))),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=((2,2))),
    keras.layers.Flatten(),
    #dense change from 4096
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    #dense change from 4096
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    #activation change from softmax
    #dense change from 2
    keras.layers.Dense(1, activation='sigmoid')
])

In [21]:
checkpoint_path = "model_checkpoints_weights/gender_model_adience_checkpoint_26nov.ckpt" # note: remember saving directly to Colab is temporary

# Create a ModelCheckpoint callback that saves the model's weights only
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         save_weights_only=True, # set to False to save the entire model
                                                         save_best_only=True, # set to True to save only the best model instead of a model every epoch 
                                                         save_freq="epoch", # save every epoch
                                                         verbose=1)

In [22]:
learning_rate = 0.01
epochs = 50
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = SGD(learning_rate=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
#loss and optimiser change from sparse and adam
model.compile(optimizer=sgd, loss = tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy', Precision() , Recall()])

### Training the Model

In [23]:
history = model.fit(ds_train, validation_data=ds_test, epochs=epochs, callbacks = [checkpoint_callback])

Epoch 1/50


UnknownError: Graph execution error:

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\photo\src\denoising.cpp:178: error: (-5:Bad argument) Type of input image should be CV_8UC3 or CV_8UC4! in function 'cv::fastNlMeansDenoisingColored'

Traceback (most recent call last):

  File "e:\virtualenv\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "e:\virtualenv\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "e:\virtualenv\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1035, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\USER\AppData\Local\Temp/ipykernel_468/3564878358.py", line 9, in yield_training_values
    denoised_image = cv2.fastNlMeansDenoisingColored(img, None, 5, 6, 7, 21)

cv2.error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\photo\src\den

In [ ]:
# Plot the validation and training data separately
def plot_loss_curves(history):
    """
    Returns separate loss curves for training and validation metrics.
    """ 
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    epochs = range(len(history.history['loss']))

    # Plot loss
    plt.plot(epochs, loss, label='training_loss')
    plt.plot(epochs, val_loss, label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label='training_accuracy')
    plt.plot(epochs, val_accuracy, label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();

## Saving the Model

In [ ]:
model.save("gender_26nov_model.h5") 

In [ ]:
plot_loss_curves(history)

In [ ]:
model.evaluate(ds_test)

In [ ]:
model.summary()